In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-05-26 16:00:00')


In [2]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/49323
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-04-28 16:00:00 to 2023-05-26 15:00:00


2023-05-26 09:55:36,621 INFO: USE `taxi_demand_alvin_featurestore`
2023-05-26 09:55:37,197 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_alvin_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-04-27 04:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-05-27 03:00:00.000'


AssertionError: Time-series data is not complete. Make sure your feature pipeline is up and runnning.

In [ ]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['pickup_hour'] = current_date
predictions

### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})